# Task for Today  

***

## Food Image Classification  

Given *images of 101 different foods*, let's try to classifify the **food** present in a given image.

We will use a TensorFlow/Keras pretrained MobileNetV2 CNN to make our predictions.

# Getting Started

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

In [3]:
image_dir = Path('../input/food41/images')
test_image_dir = Path('../input/fooddataset/Food')

# Creating File DataFrame

In [4]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

images = pd.concat([filepaths, labels], axis=1)

category_samples = []
for category in images['Label'].unique():
    category_slice = images.query("Label == @category")
    category_samples.append(category_slice.sample(100, random_state=1))
image_df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [4]:
image_df

,Filepath,Label
0,../input/food41/images/greek_salad/1330935.jpg,greek_salad
1,../input/food41/images/pancakes/3562559.jpg,pancakes
2,../input/food41/images/beef_carpaccio/484148.jpg,beef_carpaccio
3,../input/food41/images/french_fries/421038.jpg,french_fries
4,../input/food41/images/bread_pudding/689770.jpg,bread_pudding
...,...,...
10095,../input/food41/images/shrimp_and_grits/149216...,shrimp_and_grits
10096,../input/food41/images/sushi/2674024.jpg,sushi
10097,../input/food41/images/poutine/508222.jpg,poutine
10098,../input/food41/images/paella/265807.jpg,paella


In [55]:
filepaths_test = list(test_image_dir.glob(r'**/*.jpg*'))
filepaths_test.extend(list(test_image_dir.glob(r'**/*.png*')))
filepaths_test = pd.Series(filepaths_test, name='Filepath').astype(str)
image_test_df = pd.concat([filepaths_test], axis=1)

In [56]:
image_test_df

,Filepath
0,../input/fooddataset/Food/20170521_091328.jpg
1,../input/fooddataset/Food/IMG_5028.jpg
2,../input/fooddataset/Food/IMG_9594.jpg
3,../input/fooddataset/Food/IMG_7895.jpg
4,../input/fooddataset/Food/marine iguana huntin...
...,...
663,../input/fooddataset/Food/IMG_7004.png
664,../input/fooddataset/Food/IMG_7003.png
665,../input/fooddataset/Food/IMG_5940.png
666,../input/fooddataset/Food/IMG_7007.png


In [7]:
image_df['Label'].value_counts()

gnocchi           100
pho               100
club_sandwich     100
eggs_benedict     100
clam_chowder      100
                 ... 
croque_madame     100
pizza             100
french_fries      100
fried_calamari    100
grilled_salmon    100
Name: Label, Length: 101, dtype: int64

# Train-Test Split

In [57]:
train_df = image_df
test_df = image_test_df

# Creating Generators

In [9]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [58]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col=None,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode=None,
    batch_size=32,
    shuffle=False,
    validate_filenames=False
)

Found 8080 validated image filenames belonging to 101 classes.
Found 2020 validated image filenames belonging to 101 classes.
Found 668 non-validated image filenames.


# Modeling

In [36]:
pretrained_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

In [12]:
inputs = pretrained_model.input

x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(101, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

# Training

In [13]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
253/253 [==============================] - 122s 456ms/step - loss: 4.0965 - accuracy: 0.0999 - val_loss: 2.6693 - val_accuracy: 0.3505
Epoch 2/100
253/253 [==============================] - 53s 210ms/step - loss: 2.2332 - accuracy: 0.4305 - val_loss: 2.4752 - val_accuracy: 0.3911
Epoch 3/100
253/253 [==============================] - 55s 216ms/step - loss: 1.7045 - accuracy: 0.5472 - val_loss: 2.3852 - val_accuracy: 0.4158
Epoch 4/100
253/253 [==============================] - 54s 215ms/step - loss: 1.3781 - accuracy: 0.6330 - val_loss: 2.2934 - val_accuracy: 0.4441
Epoch 5/100
253/253 [==============================] - 55s 217ms/step - loss: 1.1293 - accuracy: 0.6874 - val_loss: 2.4015 - val_accuracy: 0.4366
Epoch 6/100
253/253 [==============================] - 55s 218ms/step - loss: 0.8992 - accuracy: 0.7519 - val_loss: 2.4463 - val_accuracy: 0.4520
Epoch 7/100
253/253 [==============================] - 54s 215ms/step - loss: 0.6951 - accuracy: 0.8121 - val_loss: 2.5940 

# Results

In [14]:
results = model.evaluate(test_images, verbose=0)
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

Test Accuracy: 0.00%


In [59]:
predictions = np.argmax(model.predict(test_images), axis=1)

#cm = confusion_matrix(test_images.labels, predictions)
#clr = classification_report(test_images.labels, predictions, target_names=test_images.class_indices, zero_division=0)

In [66]:
predictions

0      86
1      88
2      71
3      58
4      60
       ..
663    84
664    95
665    95
666     0
667     1
Name: Predictions, Length: 668, dtype: int64

In [69]:
for i in range(len(predictions)):
    for key, value in train_images.class_indices.items():
            if value == predictions[i]:
                predictions[i] = key
predictions

0             sashimi
1       seaweed_salad
2              paella
3           ice_cream
4      lobster_bisque
            ...      
663           risotto
664             sushi
665             sushi
666         apple_pie
667    baby_back_ribs
Name: Predictions, Length: 668, dtype: object

In [70]:
train_images.class_indices

{'apple_pie': 0,
 'baby_back_ribs': 1,
 'baklava': 2,
 'beef_carpaccio': 3,
 'beef_tartare': 4,
 'beet_salad': 5,
 'beignets': 6,
 'bibimbap': 7,
 'bread_pudding': 8,
 'breakfast_burrito': 9,
 'bruschetta': 10,
 'caesar_salad': 11,
 'cannoli': 12,
 'caprese_salad': 13,
 'carrot_cake': 14,
 'ceviche': 15,
 'cheese_plate': 16,
 'cheesecake': 17,
 'chicken_curry': 18,
 'chicken_quesadilla': 19,
 'chicken_wings': 20,
 'chocolate_cake': 21,
 'chocolate_mousse': 22,
 'churros': 23,
 'clam_chowder': 24,
 'club_sandwich': 25,
 'crab_cakes': 26,
 'creme_brulee': 27,
 'croque_madame': 28,
 'cup_cakes': 29,
 'deviled_eggs': 30,
 'donuts': 31,
 'dumplings': 32,
 'edamame': 33,
 'eggs_benedict': 34,
 'escargots': 35,
 'falafel': 36,
 'filet_mignon': 37,
 'fish_and_chips': 38,
 'foie_gras': 39,
 'french_fries': 40,
 'french_onion_soup': 41,
 'french_toast': 42,
 'fried_calamari': 43,
 'fried_rice': 44,
 'frozen_yogurt': 45,
 'garlic_bread': 46,
 'gnocchi': 47,
 'greek_salad': 48,
 'grilled_cheese_sa

In [71]:
predictionsSeries = pd.Series(predictions, name='Predictions')
predictedImages = pd.concat([test_df, predictionsSeries], axis=1)
predictedImages

,Filepath,Predictions
0,../input/fooddataset/Food/20170521_091328.jpg,sashimi
1,../input/fooddataset/Food/IMG_5028.jpg,seaweed_salad
2,../input/fooddataset/Food/IMG_9594.jpg,paella
3,../input/fooddataset/Food/IMG_7895.jpg,ice_cream
4,../input/fooddataset/Food/marine iguana huntin...,lobster_bisque
...,...,...
663,../input/fooddataset/Food/IMG_7004.png,risotto
664,../input/fooddataset/Food/IMG_7003.png,sushi
665,../input/fooddataset/Food/IMG_5940.png,sushi
666,../input/fooddataset/Food/IMG_7007.png,apple_pie


In [76]:
!cd /kaggle/working
predictedImages.to_csv('mycsvfile.csv')

In [ ]:
plt.figure(figsize=(30, 30))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cmap='Blues', cbar=False)
plt.xticks(ticks=np.arange(101) + 0.5, labels=test_images.class_indices, rotation=90)
plt.yticks(ticks=np.arange(101) + 0.5, labels=test_images.class_indices, rotation=0)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
print("Classification Report:\n----------------------\n", clr)

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/35iRdqY01co